In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import dask.dataframe as ddf
from pandas import Series, DataFrame
%matplotlib inline

In [2]:
import seaborn as sn
import plotly.express as px

### Request for Census Tract Level Population data from Census API

In [3]:
stateCode = ['01', '02', '04', '05', '06', '08', '09', '10', '11', '12', '13', '15', '16',
             '17', '18', '19', '20', '21', '22', '23', '24', '25', '26', '27', '28', '29',
            '30', '31', '32', '33', '34', '35', '36', '37', '38', '39', '40', '41', '42', 
             '44', '45', '46', '47', '48', '49', '50', '51', '53', '54', '55', '56']

In [4]:
len(stateCode)

51

In [5]:
import requests
import pandas as pd
from datetime import datetime
import time
import json
from io import StringIO

In [6]:
def collectAll_county(year, variables):
    basic_url = 'https://api.census.gov/data/' + year + '/acs/acs5?get=NAME,'+ variables + '&for=county:*&in=state:*&key=250f58e35283d400e4185b197e3abe9021079afe'
    r = requests.get(basic_url)
    x = r.json()
    print(x[0])
    variable_list = variables.split(',')
    column_list = ['county_name']
    for item in variable_list:
        column_list.append(item)
    column_list.append('state code')
    column_list.append('county code')
    df = pd.DataFrame(x[1:])
    df.columns = column_list
    return df

#### Total population: B01003_001E

In [7]:
year = '2010'
variables = 'B01003_001E'

In [8]:
pop_10 = collectAll_county(year,variables)
pop_10.head()

['NAME', 'B01003_001E', 'state', 'county']


,county_name,B01003_001E,state code,county code
0,"Manatí Municipio, Puerto Rico",44631,72,091
1,"Merced County, California",250699,06,047
2,"Modoc County, California",9605,06,049
3,"Mono County, California",13905,06,051
4,"Monterey County, California",407435,06,053


In [10]:
pop_10.shape

(3221, 4)

#### 1.per capita income: B19301_001E

In [11]:
variables = 'B19301_001E'
df1 = collectAll_county(year,variables)

['NAME', 'B19301_001E', 'state', 'county']


In [12]:
df1.shape

(3221, 4)

In [13]:
df1 = df1.rename(columns = {'B19301_001E': 'per_capita_income'})
df1['fips'] = df1['state code'] + df1['county code']
df1.head()

,county_name,per_capita_income,state code,county code,fips
0,"Manatí Municipio, Puerto Rico",8949,72,091,72091
1,"Merced County, California",18041,06,047,06047
2,"Modoc County, California",20536,06,049,06049
3,"Mono County, California",27321,06,051,06051
4,"Monterey County, California",25776,06,053,06053


#### 2.Percent of Population Below Poverty Level: (C17002_002E+C17002_003E)/C17002_001E

In [14]:
variables = 'C17002_002E,C17002_003E,C17002_001E'
df2 = collectAll_county(year,variables)

['NAME', 'C17002_002E', 'C17002_003E', 'C17002_001E', 'state', 'county']


In [15]:
df2['C17002_002E'] = df2['C17002_002E'].astype('int')
df2['C17002_003E'] = df2['C17002_003E'].astype('int')
df2['C17002_001E'] = df2['C17002_001E'].astype('int')

In [16]:
df2['percent_below_poverty'] = ((df2['C17002_002E'] + df2['C17002_003E'])/df2['C17002_001E'])*100

In [17]:
df2['fips'] = df2['state code'] + df2['county code']
df2.head()

,county_name,C17002_002E,C17002_003E,C17002_001E,state code,county code,percent_below_poverty,fips
0,"Manatí Municipio, Puerto Rico",12438,9234,44247,72,091,48.979592,72091
1,"Merced County, California",20935,32803,246260,06,047,21.821652,06047
2,"Modoc County, California",647,1062,9268,06,049,18.439793,06049
3,"Mono County, California",447,1217,13853,06,051,12.011839,06051
4,"Monterey County, California",18752,35524,389514,06,053,13.934287,06053


#### 3.Percent of Population 25+ with Less than a 12th Grade Education: (B15002_003E,B15002_004E,B15002_005E,B15002_003E,B15002_006E,B15002_007E,B15002_008E,B15002_009E,B15002_010E,B15002_020E,B15002_021E,B15002_022E,B15002_023E,B15002_024E,B15002_025E,B15002_026E,B15002_027E)/B15003_001E

In [18]:
def collectAll_county2(year, variables):
    basic_url = 'https://api.census.gov/data/' + year + '/acs/acs1?get=NAME,'+ variables + '&for=county:*&in=state:*'
    r = requests.get(basic_url)
    x = r.json()
    print(x[0])
    variable_list = variables.split(',')
    column_list = ['county_name']
    for item in variable_list:
        column_list.append(item)
    column_list.append('state code')
    column_list.append('county code')
    df = pd.DataFrame(x[1:])
    df.columns = column_list
    return df

In [19]:
variables = 'B15002_003E,B15002_004E,B15002_005E,B15002_006E,B15002_007E,B15002_008E,B15002_009E,B15002_010E,B15002_020E,B15002_021E,B15002_022E,B15002_023E,B15002_024E,B15002_025E,B15002_026E,B15002_027E,B15002_001E'
df3 = collectAll_county(year,variables)

['NAME', 'B15002_003E', 'B15002_004E', 'B15002_005E', 'B15002_006E', 'B15002_007E', 'B15002_008E', 'B15002_009E', 'B15002_010E', 'B15002_020E', 'B15002_021E', 'B15002_022E', 'B15002_023E', 'B15002_024E', 'B15002_025E', 'B15002_026E', 'B15002_027E', 'B15002_001E', 'state', 'county']


In [20]:
df3.shape

(3221, 20)

In [21]:
def changeInteger(value):
    if value != None:
        return int(value)
    else:
        return np.nan

In [22]:
df3['B15002_003E'] = df3['B15002_003E'].apply(changeInteger)
df3['B15002_004E'] = df3['B15002_004E'].apply(changeInteger)
df3['B15002_005E'] = df3['B15002_005E'].apply(changeInteger)
df3['B15002_006E'] = df3['B15002_006E'].apply(changeInteger)
df3['B15002_007E'] = df3['B15002_007E'].apply(changeInteger)
df3['B15002_008E'] = df3['B15002_008E'].apply(changeInteger)
df3['B15002_009E'] = df3['B15002_009E'].apply(changeInteger)
df3['B15002_010E'] = df3['B15002_010E'].apply(changeInteger)
df3['B15002_020E'] = df3['B15002_020E'].apply(changeInteger)
df3['B15002_021E'] = df3['B15002_021E'].apply(changeInteger)
df3['B15002_022E'] = df3['B15002_022E'].apply(changeInteger)
df3['B15002_023E'] = df3['B15002_023E'].apply(changeInteger)
df3['B15002_024E'] = df3['B15002_024E'].apply(changeInteger)
df3['B15002_025E'] = df3['B15002_025E'].apply(changeInteger)
df3['B15002_026E'] = df3['B15002_026E'].apply(changeInteger)
df3['B15002_027E'] = df3['B15002_027E'].apply(changeInteger)
df3['B15002_001E'] = df3['B15002_001E'].apply(changeInteger)

In [23]:
df3['percent_no_highschool_diploma'] = ((df3['B15002_003E']+df3['B15002_004E']+df3['B15002_005E']+df3['B15002_006E']+df3['B15002_007E']+df3['B15002_008E']+df3['B15002_009E']
                                       +df3['B15002_010E']+df3['B15002_020E']+df3['B15002_021E']+df3['B15002_022E']+df3['B15002_023E']+df3['B15002_024E']+df3['B15002_025E']+df3['B15002_003E']
                                       +df3['B15002_026E']+df3['B15002_027E'])/df3['B15002_001E'])*100

In [24]:
df3['fips'] = df3['state code'] + df3['county code']
df3.head()

,county_name,B15002_003E,B15002_004E,B15002_005E,B15002_006E,B15002_007E,B15002_008E,B15002_009E,B15002_010E,B15002_020E,...,B15002_023E,B15002_024E,B15002_025E,B15002_026E,B15002_027E,B15002_001E,state code,county code,percent_no_highschool_diploma,fips
0,"Manatí Municipio, Puerto Rico",561,1055,1044,855,562,222,439,210,483,...,948,763,198,513,113,28686,72,091,38.130098,72091
1,"Merced County, California",2860,3018,6196,2495,2001,1775,2704,2738,3533,...,2636,1952,1963,2319,2591,140880,06,047,34.993612,06047
2,"Modoc County, California",23,18,52,86,53,125,72,132,17,...,59,6,85,93,117,6854,06,049,16.049022,06049
3,"Mono County, California",9,71,24,135,41,106,67,201,75,...,51,92,17,65,62,9297,06,051,12.670754,06051
4,"Monterey County, California",3898,5471,11361,4213,3960,3364,4559,3565,3573,...,3550,2734,2028,3156,2433,252221,06,053,30.863013,06053


#### 4.Percent of Population Living in Mobile Homes: (B25033_006E+B25033_007E+B25033_012E+B25033_013E)/B25033_001E

In [25]:
variables = 'B25033_006E,B25033_007E,B25033_012E,B25033_013E,B25033_001E'
df4 = collectAll_county(year,variables)

['NAME', 'B25033_006E', 'B25033_007E', 'B25033_012E', 'B25033_013E', 'B25033_001E', 'state', 'county']


In [26]:
df4.shape

(3221, 8)

In [27]:
df4['B25033_006E'] = df4['B25033_006E'].apply(changeInteger)
df4['B25033_007E'] = df4['B25033_007E'].apply(changeInteger)
df4['B25033_012E'] = df4['B25033_012E'].apply(changeInteger)
df4['B25033_013E'] = df4['B25033_013E'].apply(changeInteger)
df4['B25033_001E'] = df4['B25033_001E'].apply(changeInteger)

In [28]:
df4['percent_mobile_homes'] = ((df4['B25033_006E'] + df4['B25033_007E'] + df4['B25033_012E'] + df4['B25033_013E'])/df4['B25033_001E'])*100

In [29]:
df4['fips'] = df4['state code'] + df4['county code']
df4.head()

,county_name,B25033_006E,B25033_007E,B25033_012E,B25033_013E,B25033_001E,state code,county code,percent_mobile_homes,fips
0,"Manatí Municipio, Puerto Rico",79,0,13,0,44468,72,091,0.206890,72091
1,"Merced County, California",6843,37,5656,21,247151,06,047,5.080700,06047
2,"Modoc County, California",902,11,493,0,9282,06,049,15.147598,06049
3,"Mono County, California",1206,0,447,0,13789,06,051,11.987816,06051
4,"Monterey County, California",7834,145,7557,72,389956,06,053,4.002503,06053


#### 5.Percent of Population with No Vehicle Available: (B25044_003E+B25044_010E)/B25044_001E

In [30]:
variables = 'B25044_003E,B25044_010E,B25044_001E'
df5 = collectAll_county(year,variables)

['NAME', 'B25044_003E', 'B25044_010E', 'B25044_001E', 'state', 'county']


In [31]:
df5.shape

(3221, 6)

In [32]:
df5['B25044_003E'] = df5['B25044_003E'].apply(changeInteger)
df5['B25044_010E'] = df5['B25044_010E'].apply(changeInteger)
df5['B25044_001E'] = df5['B25044_001E'].apply(changeInteger)

In [33]:
df5['percent_no_vehicle'] = ((df5['B25044_003E'] + df5['B25044_010E'])/df5['B25044_001E'])*100

In [34]:
df5['fips'] = df5['state code'] + df5['county code']
df5.head()

,county_name,B25044_003E,B25044_010E,B25044_001E,state code,county code,percent_no_vehicle,fips
0,"Manatí Municipio, Puerto Rico",2197,1487,15363,72,091,23.979691,72091
1,"Merced County, California",1338,4476,73586,06,047,7.900959,06047
2,"Modoc County, California",66,237,3977,06,049,7.618808,06049
3,"Mono County, California",19,264,5283,06,051,5.356805,06051
4,"Monterey County, California",1822,6128,124963,06,053,6.361883,06053


#### 6.Percent of Population Unemployed:sum(B21005_006E, B21005_007E, B21005_011E,B21005_012E,B21005_017E,B21005_022E,B21005_028E,B21005_029E,B21005_033E,B21005_034E)/B21005_001E

In [35]:
variables = 'B21005_006E,B21005_007E,B21005_011E,B21005_012E,B21005_017E,B21005_022E,B21005_028E,B21005_029E,B21005_033E,B21005_034E,B21005_001E'
df6 = collectAll_county(year,variables)

['NAME', 'B21005_006E', 'B21005_007E', 'B21005_011E', 'B21005_012E', 'B21005_017E', 'B21005_022E', 'B21005_028E', 'B21005_029E', 'B21005_033E', 'B21005_034E', 'B21005_001E', 'state', 'county']


In [36]:
df6.shape

(3221, 14)

In [37]:
df6['B21005_006E'] = df6['B21005_006E'].apply(changeInteger)
df6['B21005_007E'] = df6['B21005_007E'].apply(changeInteger)
df6['B21005_011E'] = df6['B21005_011E'].apply(changeInteger)
df6['B21005_012E'] = df6['B21005_012E'].apply(changeInteger)
df6['B21005_017E'] = df6['B21005_017E'].apply(changeInteger)
df6['B21005_022E'] = df6['B21005_022E'].apply(changeInteger)
df6['B21005_028E'] = df6['B21005_028E'].apply(changeInteger)
df6['B21005_029E'] = df6['B21005_029E'].apply(changeInteger)
df6['B21005_033E'] = df6['B21005_033E'].apply(changeInteger)
df6['B21005_034E'] = df6['B21005_034E'].apply(changeInteger)
df6['B21005_001E'] = df6['B21005_001E'].apply(changeInteger)

In [38]:
df6['percent_unemployed_CDC'] = ((df6['B21005_006E']+df6['B21005_007E']+df6['B21005_011E']+df6['B21005_012E']+df6['B21005_017E']+df6['B21005_022E']+df6['B21005_028E']+df6['B21005_029E']+df6['B21005_033E']+df6['B21005_034E'])/df6['B21005_001E'])*100

In [39]:
df6['fips'] = df6['state code'] + df6['county code']
df6.head()

,county_name,B21005_006E,B21005_007E,B21005_011E,B21005_012E,B21005_017E,B21005_022E,B21005_028E,B21005_029E,B21005_033E,B21005_034E,B21005_001E,state code,county code,percent_unemployed_CDC,fips
0,"Manatí Municipio, Puerto Rico",17,0,2237,3345,54,1088,0,216,113,3607,26541,72,091,40.228326,72091
1,"Merced County, California",180,136,8354,17737,267,4586,61,1173,815,8227,147277,06,047,28.202639,06047
2,"Modoc County, California",0,15,220,452,39,153,7,170,47,567,5664,06,049,29.484463,06049
3,"Mono County, California",0,0,285,320,13,166,0,40,63,526,9600,06,051,14.718750,06051
4,"Monterey County, California",145,242,10489,24649,191,6480,332,2123,1202,11691,249381,06,053,23.074733,06053


#### 7.Percent of Population Living In Accommodations with Less Than 1 Room Per Person/Crowding: (B25014_005E+B25014_006E+B25014_007E+B25014_011E+B25014_012E+B25014_013E)/B25014_001E

In [40]:
variables = 'B25014_005E,B25014_006E,B25014_007E,B25014_011E,B25014_012E,B25014_013E,B25014_001E'
df7 = collectAll_county(year,variables)

['NAME', 'B25014_005E', 'B25014_006E', 'B25014_007E', 'B25014_011E', 'B25014_012E', 'B25014_013E', 'B25014_001E', 'state', 'county']


In [41]:
df7.shape

(3221, 10)

In [42]:
df7['B25014_005E'] = df7['B25014_005E'].apply(changeInteger)
df7['B25014_006E'] = df7['B25014_006E'].apply(changeInteger)
df7['B25014_007E'] = df7['B25014_007E'].apply(changeInteger)
df7['B25014_011E'] = df7['B25014_011E'].apply(changeInteger)
df7['B25014_012E'] = df7['B25014_012E'].apply(changeInteger)
df7['B25014_013E'] = df7['B25014_013E'].apply(changeInteger)
df7['B25014_001E'] = df7['B25014_001E'].apply(changeInteger)

In [43]:
df7['percent_overcrowding'] = ((df7['B25014_005E']+df7['B25014_006E']+df7['B25014_007E']+df7['B25014_011E']+df7['B25014_012E']+df7['B25014_013E'])/df7['B25014_001E'])*100

In [44]:
df7['fips'] = df7['state code'] + df7['county code']
df7.head()

,county_name,B25014_005E,B25014_006E,B25014_007E,B25014_011E,B25014_012E,B25014_013E,B25014_001E,state code,county code,percent_overcrowding,fips
0,"Manatí Municipio, Puerto Rico",634,110,39,378,136,41,15363,72,091,8.709236,72091
1,"Merced County, California",1637,313,63,3124,996,321,73586,06,047,8.770690,06047
2,"Modoc County, California",20,11,0,73,18,0,3977,06,049,3.067639,06049
3,"Mono County, California",0,0,0,147,117,14,5283,06,051,5.262162,06051
4,"Monterey County, California",2914,892,97,6093,2885,727,124963,06,053,10.889623,06053


#### 8.Percent of Housing Units with 10+ Units in Structure: (B25024_007E+B25024_008E+B25024_009E)/B25024_001E

In [45]:
variables = 'B25024_007E,B25024_008E,B25024_009E,B25024_001E'
df8 = collectAll_county(year,variables)

['NAME', 'B25024_007E', 'B25024_008E', 'B25024_009E', 'B25024_001E', 'state', 'county']


In [46]:
df8.shape

(3221, 7)

In [47]:
df8['B25024_007E'] = df8['B25024_007E'].apply(changeInteger)
df8['B25024_008E'] = df8['B25024_008E'].apply(changeInteger)
df8['B25024_009E'] = df8['B25024_009E'].apply(changeInteger)
df8['B25024_001E'] = df8['B25024_001E'].apply(changeInteger)

In [48]:
df8['percent_multi_unit_housing'] = ((df8['B25024_007E']+df8['B25024_008E']+df8['B25024_009E'])/df8['B25024_001E'])*100

In [49]:
df8['fips'] = df8['state code'] + df8['county code']
df8.head()

,county_name,B25024_007E,B25024_008E,B25024_009E,B25024_001E,state code,county code,percent_multi_unit_housing,fips
0,"Manatí Municipio, Puerto Rico",550,37,29,17605,72,091,3.499006,72091
1,"Merced County, California",1418,837,1173,83089,06,047,4.125697,06047
2,"Modoc County, California",28,71,18,5133,06,049,2.279369,06049
3,"Mono County, California",1228,954,196,13802,06,051,17.229387,06051
4,"Monterey County, California",6071,3718,3526,138833,06,053,9.590659,06053


#### 9.Percent Of Population 65+: (B01001_020E,B01001_021E,B01001_022E,B01001_023E,B01001_024E,B01001_025E,B01001_044E,B01001_045E,B01001_046E,B01001_047E,B01001_049E)/B01001_001E

In [50]:
variables = 'B01001_020E,B01001_021E,B01001_022E,B01001_023E,B01001_024E,B01001_025E,B01001_044E,B01001_045E,B01001_046E,B01001_047E,B01001_048E,B01001_049E,B01001_001E'
df9 = collectAll_county(year,variables)

['NAME', 'B01001_020E', 'B01001_021E', 'B01001_022E', 'B01001_023E', 'B01001_024E', 'B01001_025E', 'B01001_044E', 'B01001_045E', 'B01001_046E', 'B01001_047E', 'B01001_048E', 'B01001_049E', 'B01001_001E', 'state', 'county']


In [51]:
df9.shape

(3221, 16)

In [52]:
df9['B01001_020E'] = df9['B01001_020E'].apply(changeInteger)
df9['B01001_021E'] = df9['B01001_021E'].apply(changeInteger)
df9['B01001_022E'] = df9['B01001_022E'].apply(changeInteger)
df9['B01001_023E'] = df9['B01001_023E'].apply(changeInteger)
df9['B01001_024E'] = df9['B01001_024E'].apply(changeInteger)
df9['B01001_025E'] = df9['B01001_025E'].apply(changeInteger)
df9['B01001_044E'] = df9['B01001_044E'].apply(changeInteger)
df9['B01001_045E'] = df9['B01001_045E'].apply(changeInteger)
df9['B01001_046E'] = df9['B01001_046E'].apply(changeInteger)
df9['B01001_047E'] = df9['B01001_047E'].apply(changeInteger)
df9['B01001_048E'] = df9['B01001_048E'].apply(changeInteger)
df9['B01001_049E'] = df9['B01001_049E'].apply(changeInteger)
df9['B01001_001E'] = df9['B01001_001E'].apply(changeInteger)

In [53]:
df9['percent_age_65_and_older'] = ((df9['B01001_020E']+df9['B01001_021E']+df9['B01001_022E']+df9['B01001_023E']+df9['B01001_024E']+df9['B01001_025E']+df9['B01001_044E']
                                  +df9['B01001_045E']+df9['B01001_046E']+df9['B01001_047E']+df9['B01001_048E']+df9['B01001_049E'])/df9['B01001_001E'])*100

In [54]:
df9['fips'] = df9['state code'] + df9['county code']
df9.head()

,county_name,B01001_020E,B01001_021E,B01001_022E,B01001_023E,B01001_024E,B01001_025E,B01001_044E,B01001_045E,B01001_046E,B01001_047E,B01001_048E,B01001_049E,B01001_001E,state code,county code,percent_age_65_and_older,fips
0,"Manatí Municipio, Puerto Rico",472,476,718,596,323,218,478,790,796,739,504,256,44631,72,091,14.263628,72091
1,"Merced County, California",1507,1749,2537,1903,1183,1038,1507,2245,2920,2511,1804,1732,250699,06,047,9.029154,06047
2,"Modoc County, California",67,273,204,159,105,65,230,213,91,169,116,120,9605,06,049,18.865174,06049
3,"Mono County, California",143,158,91,112,86,11,100,192,116,58,24,66,13905,06,051,8.320748,06051
4,"Monterey County, California",2712,3178,4216,3634,2671,2068,2737,3297,5307,4816,3935,3984,407435,06,053,10.444611,06053


#### 10.Percent Of Population Who Do Not Speak English: B99163_005E/B99163_001E

In [55]:
variables = 'B99163_005E,B99163_001E'
df10 = collectAll_county(year,variables)

['NAME', 'B99163_005E', 'B99163_001E', 'state', 'county']


In [56]:
df10.shape

(3221, 5)

In [57]:
df10['B99163_005E'] = df10['B99163_005E'].apply(changeInteger)
df10['B99163_001E'] = df10['B99163_001E'].apply(changeInteger)

In [58]:
df10['percent_limited_english_abilities'] = (df10['B99163_005E']/df10['B99163_001E'])*100

In [59]:
df10['fips'] = df10['state code'] + df10['county code']
df10.head()

,county_name,B99163_005E,B99163_001E,state code,county code,percent_limited_english_abilities,fips
0,"Manatí Municipio, Puerto Rico",39250,41750,72,091,94.011976,72091
1,"Merced County, California",114706,228318,06,047,50.239578,06047
2,"Modoc County, California",1213,9093,06,049,13.339932,06049
3,"Mono County, California",3058,12992,06,051,23.537562,06051
4,"Monterey County, California",188632,374891,06,053,50.316492,06053


#### 11.Percent of Population 17 Years of Age and Under: (B01001_003E+B01001_004E+B01001_005E+B01001_006E+B01001_027E+B01001_028E+B01001_029E+B01001_030E)/B01003_001E

In [60]:
variables = 'B01001_003E,B01001_004E,B01001_005E,B01001_006E,B01001_027E,B01001_028E,B01001_029E,B01001_030E,B01003_001E'
df11 = collectAll_county(year,variables)

['NAME', 'B01001_003E', 'B01001_004E', 'B01001_005E', 'B01001_006E', 'B01001_027E', 'B01001_028E', 'B01001_029E', 'B01001_030E', 'B01003_001E', 'state', 'county']


In [61]:
df11.shape

(3221, 12)

In [62]:
df11['B01001_003E'] = df11['B01001_003E'].apply(changeInteger)
df11['B01001_004E'] = df11['B01001_004E'].apply(changeInteger)
df11['B01001_005E'] = df11['B01001_005E'].apply(changeInteger)
df11['B01001_006E'] = df11['B01001_006E'].apply(changeInteger)
df11['B01001_027E'] = df11['B01001_027E'].apply(changeInteger)
df11['B01001_028E'] = df11['B01001_028E'].apply(changeInteger)
df11['B01001_029E'] = df11['B01001_029E'].apply(changeInteger)
df11['B01001_030E'] = df11['B01001_030E'].apply(changeInteger)
df11['B01003_001E'] = df11['B01003_001E'].apply(changeInteger)

In [63]:
df11['percent_age_17_and_younger'] = ((df11['B01001_003E']+df11['B01001_004E']+df11['B01001_005E']+df11['B01001_006E']+df11['B01001_027E']+df11['B01001_028E']+df11['B01001_029E']+df11['B01001_030E'])/df11['B01003_001E'])*100

In [64]:
df11['fips'] = df11['state code'] + df11['county code']
df11.head()

,county_name,B01001_003E,B01001_004E,B01001_005E,B01001_006E,B01001_027E,B01001_028E,B01001_029E,B01001_030E,B01003_001E,state code,county code,percent_age_17_and_younger,fips
0,"Manatí Municipio, Puerto Rico",1508,1837,1654,1068,1373,1360,1868,1014,44631,72,091,26.174632,72091
1,"Merced County, California",11511,11200,11359,7360,10870,10802,10655,6856,250699,06,047,32.155294,06047
2,"Modoc County, California",262,277,334,213,250,319,307,167,9605,06,049,22.165539,06049
3,"Mono County, California",413,143,654,354,500,428,353,127,13905,06,051,21.373607,06051
4,"Monterey County, California",16683,14707,15142,9700,15861,14456,13739,9201,407435,06,053,26.872753,06053


#### 12.Percent Minority: 1-(B03002_003E/B03002_001E)

In [65]:
variables = 'B03002_003E,B03002_001E'
df12 = collectAll_county(year,variables)

['NAME', 'B03002_003E', 'B03002_001E', 'state', 'county']


In [66]:
df12.shape

(3221, 5)

In [67]:
df12['B03002_003E'] = df12['B03002_003E'].apply(changeInteger)
df12['B03002_001E'] = df12['B03002_001E'].apply(changeInteger)

In [68]:
df12['percent_minorities'] = (1 - (df12['B03002_003E']/df12['B03002_001E']))*100

In [69]:
df12['fips'] = df12['state code'] + df12['county code']
df12.head()

,county_name,B03002_003E,B03002_001E,state code,county code,percent_minorities,fips
0,"Manatí Municipio, Puerto Rico",292,44631,72,091,99.345746,72091
1,"Merced County, California",82907,250699,06,047,66.929665,06047
2,"Modoc County, California",7610,9605,06,049,20.770432,06049
3,"Mono County, California",9644,13905,06,051,30.643653,06051
4,"Monterey County, California",139535,407435,06,053,65.752819,06053


#### 13.Percent of Population with a Disability: (B18101_004E+B18101_007E+B18101_010E+B18101_013E+B18101_016E+B18101_019E)/B18101_002E + (B18101_023E+B18101_026E+B18101_029E+B18101_032E+B18101_035E+B18101_038E)/B18101_021E

In [71]:
variables = 'B18101_004E,B18101_007E,B18101_010E,B18101_013E,B18101_016E,B18101_019E,B18101_002E,B18101_023E,B18101_026E,B18101_029E,B18101_032E,B18101_035E,B18101_038E,B18101_021E'
df13 = collectAll_county2(year,variables)

['NAME', 'B18101_004E', 'B18101_007E', 'B18101_010E', 'B18101_013E', 'B18101_016E', 'B18101_019E', 'B18101_002E', 'B18101_023E', 'B18101_026E', 'B18101_029E', 'B18101_032E', 'B18101_035E', 'B18101_038E', 'B18101_021E', 'state', 'county']


In [72]:
df13.shape

(818, 17)

In [73]:
df13['B18101_004E'] = df13['B18101_004E'].apply(changeInteger)
df13['B18101_007E'] = df13['B18101_007E'].apply(changeInteger)
df13['B18101_010E'] = df13['B18101_010E'].apply(changeInteger)
df13['B18101_013E'] = df13['B18101_013E'].apply(changeInteger)
df13['B18101_016E'] = df13['B18101_016E'].apply(changeInteger)
df13['B18101_002E'] = df13['B18101_002E'].apply(changeInteger)
df13['B18101_023E'] = df13['B18101_023E'].apply(changeInteger)
df13['B18101_026E'] = df13['B18101_026E'].apply(changeInteger)
df13['B18101_029E'] = df13['B18101_029E'].apply(changeInteger)
df13['B18101_032E'] = df13['B18101_032E'].apply(changeInteger)
df13['B18101_035E'] = df13['B18101_035E'].apply(changeInteger)
df13['B18101_038E'] = df13['B18101_038E'].apply(changeInteger)
df13['B18101_021E'] = df13['B18101_021E'].apply(changeInteger)

In [74]:
df13['percent_disabled'] = ((df13['B18101_004E']+df13['B18101_007E']+df13['B18101_010E']+df13['B18101_013E']+df13['B18101_016E'])/df13['B18101_002E'] + (df13['B18101_023E']+df13['B18101_026E']+df13['B18101_029E']+df13['B18101_032E']+df13['B18101_035E']+df13['B18101_038E'])/df13['B18101_021E'])*100

In [75]:
df13['fips'] = df13['state code'] + df13['county code']
df13.head()

,county_name,B18101_004E,B18101_007E,B18101_010E,B18101_013E,B18101_016E,B18101_019E,B18101_002E,B18101_023E,B18101_026E,B18101_029E,B18101_032E,B18101_035E,B18101_038E,B18101_021E,state code,county code,percent_disabled,fips
0,"Stark County, Ohio",111,2737,2196,8820,3296,5073,180825,181,1384,1939,10718,2945,7353,190642,39,151,22.351642,39151
1,"Summit County, Ohio",0,3794,3670,12755,4313,7547,257382,0,1633,4070,13499,6287,11341,277217,39,153,22.816979,39153
2,"Trumbull County, Ohio",0,1081,1878,6803,2475,3144,99547,48,923,924,6307,2576,4696,107397,39,155,26.700910,39155
3,"Tuscarawas County, Ohio",0,236,452,2863,872,1257,44336,0,12,267,2141,761,1376,46499,39,157,19.776302,39157
4,"Warren County, Ohio",0,1419,1120,3846,904,1956,101741,0,776,668,4747,1988,2254,104671,39,165,17.131692,39165


#### 14.Percent of Population Living in Group Quarters: B26001_001E/B01003_001E

In [76]:
variables = 'B26001_001E,B01003_001E'
df14 = collectAll_county(year,variables)

['NAME', 'B26001_001E', 'B01003_001E', 'state', 'county']


In [77]:
df14.shape

(3221, 5)

In [78]:
df14['B26001_001E'] = df14['B26001_001E'].apply(changeInteger)
df14['B01003_001E'] = df14['B01003_001E'].apply(changeInteger)

In [79]:
df14['percent_institutionalized_in_group_quarters'] = (df14['B26001_001E']/df14['B01003_001E'])*100

In [80]:
df14['fips'] = df14['state code'] + df14['county code']
df14.head()

,county_name,B26001_001E,B01003_001E,state code,county code,percent_institutionalized_in_group_quarters,fips
0,"Manatí Municipio, Puerto Rico",163,44631,72,091,0.365217,72091
1,"Merced County, California",3548,250699,06,047,1.415243,06047
2,"Modoc County, California",323,9605,06,049,3.362832,06049
3,"Mono County, California",116,13905,06,051,0.834232,06051
4,"Monterey County, California",17479,407435,06,053,4.290009,06053


#### 15.Percent of Children Living in Single Parent Households: (B09008_010E+B09008_011E+B09008_012E)/B09008_001E

In [81]:
variables = 'B09008_010E,B09008_011E,B09008_012E,B09008_001E'
df15 = collectAll_county(year,variables)

['NAME', 'B09008_010E', 'B09008_011E', 'B09008_012E', 'B09008_001E', 'state', 'county']


In [82]:
df15['B09008_010E'] = df15['B09008_010E'].apply(changeInteger)
df15['B09008_011E'] = df15['B09008_011E'].apply(changeInteger)
df15['B09008_012E'] = df15['B09008_012E'].apply(changeInteger)
df15['B09008_001E'] = df15['B09008_001E'].apply(changeInteger)

In [83]:
df15['percent_single_parent_households_CDC'] = ((df15['B09008_010E'] + df15['B09008_011E']+df15['B09008_012E'])/df15['B09008_001E'])*100

In [84]:
df15['fips'] = df15['state code'] + df15['county code']
df15.head()

,county_name,B09008_010E,B09008_011E,B09008_012E,B09008_001E,state code,county code,percent_single_parent_households_CDC,fips
0,"Manatí Municipio, Puerto Rico",280,4687,11,11657,72,091,42.703955,72091
1,"Merced County, California",2727,13593,123,80573,06,047,20.407581,06047
2,"Modoc County, California",85,481,0,2129,06,049,26.585251,06049
3,"Mono County, California",134,457,0,2972,06,051,19.885599,06051
4,"Monterey County, California",5685,17094,454,109406,06,053,21.235581,06053


### age 18 to 34
#### (B01001_007E+B01001_008E+B01001_009E+B01001_010E+B01001_011E+B01001_012E+B01001_031E+B01001_032E+B01001_033E+B01001_034E+B01001_035E+B01001_036E)/B01001_001E

In [85]:
variables = 'B01001_007E,B01001_008E,B01001_009E,B01001_010E,B01001_011E,B01001_012E,B01001_031E,B01001_032E,B01001_033E,B01001_034E,B01001_035E,B01001_036E,B01001_001E'
df16 = collectAll_county(year,variables)

['NAME', 'B01001_007E', 'B01001_008E', 'B01001_009E', 'B01001_010E', 'B01001_011E', 'B01001_012E', 'B01001_031E', 'B01001_032E', 'B01001_033E', 'B01001_034E', 'B01001_035E', 'B01001_036E', 'B01001_001E', 'state', 'county']


In [86]:
df16['B01001_007E'] = df16['B01001_007E'].apply(changeInteger)
df16['B01001_008E'] = df16['B01001_008E'].apply(changeInteger)
df16['B01001_009E'] = df16['B01001_009E'].apply(changeInteger)
df16['B01001_010E'] = df16['B01001_010E'].apply(changeInteger)
df16['B01001_011E'] = df16['B01001_011E'].apply(changeInteger)
df16['B01001_012E'] = df16['B01001_012E'].apply(changeInteger)
df16['B01001_031E'] = df16['B01001_031E'].apply(changeInteger)
df16['B01001_032E'] = df16['B01001_032E'].apply(changeInteger)
df16['B01001_033E'] = df16['B01001_033E'].apply(changeInteger)
df16['B01001_034E'] = df16['B01001_034E'].apply(changeInteger)
df16['B01001_035E'] = df16['B01001_035E'].apply(changeInteger)
df16['B01001_036E'] = df16['B01001_036E'].apply(changeInteger)
df16['B01001_001E'] = df16['B01001_001E'].apply(changeInteger)

In [87]:
df16['percent_age_18_to_34'] = ((df16['B01001_007E'] + df16['B01001_008E'] +df16['B01001_009E'] +df16['B01001_010E'] +df16['B01001_011E']+df16['B01001_012E']+df16['B01001_031E']+df16['B01001_032E']+df16['B01001_033E']+df16['B01001_034E']+df16['B01001_035E']+df16['B01001_036E'])/df16['B01001_001E'])*100

In [88]:
df16['fips'] = df16['state code'] + df16['county code']
df16.head()

,county_name,B01001_007E,B01001_008E,B01001_009E,B01001_010E,B01001_011E,B01001_012E,B01001_031E,B01001_032E,B01001_033E,B01001_034E,B01001_035E,B01001_036E,B01001_001E,state code,county code,percent_age_18_to_34,fips
0,"Manatí Municipio, Puerto Rico",643,391,331,696,1378,1318,679,296,317,910,1606,1478,44631,72,091,22.502297,72091
1,"Merced County, California",5050,2252,2000,5961,9255,8339,4496,2173,2041,5233,8600,8108,250699,06,047,25.332371,06047
2,"Modoc County, California",59,49,97,124,253,254,83,14,61,135,170,224,9605,06,049,15.856325,06049
3,"Mono County, California",248,282,21,441,673,618,234,26,56,328,439,197,13905,06,051,25.623876,06051
4,"Monterey County, California",7117,3470,3594,10636,17107,15758,6415,2879,2852,8762,14427,13206,407435,06,053,26.071152,06053


### Age 35 to 44
#### (B01001_013E+B01001_014E+B01001_037E+B01001_038E)/B01001_001E

In [89]:
variables = 'B01001_013E,B01001_014E,B01001_037E,B01001_038E,B01001_001E'
df17 = collectAll_county(year,variables)

['NAME', 'B01001_013E', 'B01001_014E', 'B01001_037E', 'B01001_038E', 'B01001_001E', 'state', 'county']


In [90]:
df17['B01001_013E'] = df17['B01001_013E'].apply(changeInteger)
df17['B01001_014E'] = df17['B01001_014E'].apply(changeInteger)
df17['B01001_037E'] = df17['B01001_037E'].apply(changeInteger)
df17['B01001_038E'] = df17['B01001_038E'].apply(changeInteger)
df17['B01001_001E'] = df17['B01001_001E'].apply(changeInteger)

In [91]:
df17['percent_age_35_to_44'] = ((df17['B01001_013E']+df17['B01001_014E'] +df17['B01001_037E']+df17['B01001_038E'])/df17['B01001_001E'])*100

In [92]:
df17['fips'] = df17['state code'] + df17['county code']
df17.head()

,county_name,B01001_013E,B01001_014E,B01001_037E,B01001_038E,B01001_001E,state code,county code,percent_age_35_to_44,fips
0,"Manatí Municipio, Puerto Rico",1228,1391,1554,1427,44631,72,091,12.547333,72091
1,"Merced County, California",8153,8321,8597,7525,250699,06,047,13.002046,06047
2,"Modoc County, California",305,244,226,274,9605,06,049,10.921395,06049
3,"Mono County, California",619,493,506,583,13905,06,051,15.828839,06051
4,"Monterey County, California",15264,14639,12694,12918,407435,06,053,13.625486,06053


### age 45 to 54
#### (B01001_015E+B01001_016E+B01001_039E+B01001_040E)/B01001_001E

In [93]:
variables = 'B01001_015E,B01001_016E,B01001_039E,B01001_040E,B01001_001E'
df18 = collectAll_county(year,variables)

['NAME', 'B01001_015E', 'B01001_016E', 'B01001_039E', 'B01001_040E', 'B01001_001E', 'state', 'county']


In [94]:
df18['B01001_015E'] = df18['B01001_015E'].apply(changeInteger)
df18['B01001_016E'] = df18['B01001_016E'].apply(changeInteger)
df18['B01001_039E'] = df18['B01001_039E'].apply(changeInteger)
df18['B01001_040E'] = df18['B01001_040E'].apply(changeInteger)
df18['B01001_001E'] = df18['B01001_001E'].apply(changeInteger)

In [95]:
df18['percent_age_45_to_54'] = ((df18['B01001_015E']+df18['B01001_016E'] +df18['B01001_039E']+df18['B01001_040E'])/df18['B01001_001E'])*100

In [96]:
df18['fips'] = df18['state code'] + df18['county code']
df18.head()

,county_name,B01001_015E,B01001_016E,B01001_039E,B01001_040E,B01001_001E,state code,county code,percent_age_45_to_54,fips
0,"Manatí Municipio, Puerto Rico",1298,1309,1530,1533,44631,72,091,12.704174,72091
1,"Merced County, California",7983,7134,8147,7023,250699,06,047,12.081021,06047
2,"Modoc County, California",355,324,401,366,9605,06,049,15.054659,06049
3,"Mono County, California",519,582,613,487,13905,06,051,15.828839,06051
4,"Monterey County, California",14202,13135,13004,12798,407435,06,053,13.042326,06053


### age 55 to 64
#### (B01001_017E+B01001_018E+B01001_019E+B01001_041E+B01001_042E+B01001_043E)/B01001_001E

In [97]:
variables = 'B01001_017E,B01001_018E,B01001_019E,B01001_041E,B01001_042E,B01001_043E,B01001_001E'
df19 = collectAll_county(year,variables)

['NAME', 'B01001_017E', 'B01001_018E', 'B01001_019E', 'B01001_041E', 'B01001_042E', 'B01001_043E', 'B01001_001E', 'state', 'county']


In [98]:
df19['B01001_017E'] = df19['B01001_017E'].apply(changeInteger)
df19['B01001_018E'] = df19['B01001_018E'].apply(changeInteger)
df19['B01001_019E'] = df19['B01001_019E'].apply(changeInteger)
df19['B01001_041E'] = df19['B01001_041E'].apply(changeInteger)
df19['B01001_042E'] = df19['B01001_042E'].apply(changeInteger)
df19['B01001_043E'] = df19['B01001_043E'].apply(changeInteger)
df19['B01001_001E'] = df19['B01001_001E'].apply(changeInteger)

In [99]:
df19['percent_age_55_to_64'] = ((df19['B01001_017E']+df19['B01001_018E'] +df19['B01001_019E']+df19['B01001_041E']+df19['B01001_042E']+df19['B01001_043E'])/df19['B01001_001E'])*100

In [100]:
df19['fips'] = df19['state code'] + df19['county code']
df19.head()

,county_name,B01001_017E,B01001_018E,B01001_019E,B01001_041E,B01001_042E,B01001_043E,B01001_001E,state code,county code,percent_age_55_to_64,fips
0,"Manatí Municipio, Puerto Rico",1291,505,648,1415,607,804,44631,72,091,11.807936,72091
1,"Merced County, California",6048,1874,2480,5484,2131,3042,250699,06,047,8.400113,06047
2,"Modoc County, California",423,147,223,448,199,206,9605,06,049,17.136908,06049
3,"Mono County, California",531,115,323,368,295,179,13905,06,051,13.024092,06051
4,"Monterey County, California",11075,4000,5123,11093,3710,5513,407435,06,053,9.943672,06053


### Combine 19 dataframes together

In [101]:
df = df1[['fips','per_capita_income']].merge(df2[['fips', 'percent_below_poverty']], on = 'fips', how = 'left')

In [102]:
df = df.merge(df3[['fips', 'percent_no_highschool_diploma']], on = 'fips', how = 'left')
df = df.merge(df4[['fips', 'percent_mobile_homes']], on = 'fips', how = 'left')
df = df.merge(df5[['fips', 'percent_no_vehicle']], on = 'fips', how = 'left')
df = df.merge(df6[['fips', 'percent_unemployed_CDC']], on = 'fips', how = 'left')
df = df.merge(df7[['fips', 'percent_overcrowding']], on = 'fips', how = 'left')
df = df.merge(df8[['fips', 'percent_multi_unit_housing']], on = 'fips', how = 'left')
df = df.merge(df9[['fips', 'percent_age_65_and_older']], on = 'fips', how = 'left')
df = df.merge(df10[['fips', 'percent_limited_english_abilities']], on = 'fips', how = 'left')
df = df.merge(df11[['fips', 'percent_age_17_and_younger']], on = 'fips', how = 'left')
df = df.merge(df12[['fips', 'percent_minorities']], on = 'fips', how = 'left')
df = df.merge(df13[['fips', 'percent_disabled']], on = 'fips', how = 'left')
df = df.merge(df14[['fips', 'percent_institutionalized_in_group_quarters']], on = 'fips', how = 'left')
df = df.merge(df15[['fips', 'percent_single_parent_households_CDC']], on = 'fips', how = 'left')
df = df.merge(df16[['fips', 'percent_age_18_to_34']], on = 'fips', how = 'left')
df = df.merge(df17[['fips', 'percent_age_35_to_44']], on = 'fips', how = 'left')
df = df.merge(df18[['fips', 'percent_age_45_to_54']], on = 'fips', how = 'left')
df = df.merge(df19[['fips', 'percent_age_55_to_64']], on = 'fips', how = 'left')
df.head()

,fips,per_capita_income,percent_below_poverty,percent_no_highschool_diploma,percent_mobile_homes,percent_no_vehicle,percent_unemployed_CDC,percent_overcrowding,percent_multi_unit_housing,percent_age_65_and_older,percent_limited_english_abilities,percent_age_17_and_younger,percent_minorities,percent_disabled,percent_institutionalized_in_group_quarters,percent_single_parent_households_CDC,percent_age_18_to_34,percent_age_35_to_44,percent_age_45_to_54,percent_age_55_to_64
0,72091,8949,48.979592,38.130098,0.206890,23.979691,40.228326,8.709236,3.499006,14.263628,94.011976,26.174632,99.345746,NaN,0.365217,42.703955,22.502297,12.547333,12.704174,11.807936
1,06047,18041,21.821652,34.993612,5.080700,7.900959,28.202639,8.770690,4.125697,9.029154,50.239578,32.155294,66.929665,26.952271,1.415243,20.407581,25.332371,13.002046,12.081021,8.400113
2,06049,20536,18.439793,16.049022,15.147598,7.618808,29.484463,3.067639,2.279369,18.865174,13.339932,22.165539,20.770432,NaN,3.362832,26.585251,15.856325,10.921395,15.054659,17.136908
3,06051,27321,12.011839,12.670754,11.987816,5.356805,14.718750,5.262162,17.229387,8.320748,23.537562,21.373607,30.643653,NaN,0.834232,19.885599,25.623876,15.828839,15.828839,13.024092
4,06053,25776,13.934287,30.863013,4.002503,6.361883,23.074733,10.889623,9.590659,10.444611,50.316492,26.872753,65.752819,12.794056,4.290009,21.235581,26.071152,13.625486,13.042326,9.943672


In [103]:
df['percent_age_18_to_64'] = 100 - df['percent_age_17_and_younger'] - df['percent_age_65_and_older']

In [104]:
df.head()

,fips,per_capita_income,percent_below_poverty,percent_no_highschool_diploma,percent_mobile_homes,percent_no_vehicle,percent_unemployed_CDC,percent_overcrowding,percent_multi_unit_housing,percent_age_65_and_older,...,percent_age_17_and_younger,percent_minorities,percent_disabled,percent_institutionalized_in_group_quarters,percent_single_parent_households_CDC,percent_age_18_to_34,percent_age_35_to_44,percent_age_45_to_54,percent_age_55_to_64,percent_age_18_to_64
0,72091,8949,48.979592,38.130098,0.206890,23.979691,40.228326,8.709236,3.499006,14.263628,...,26.174632,99.345746,NaN,0.365217,42.703955,22.502297,12.547333,12.704174,11.807936,59.561740
1,06047,18041,21.821652,34.993612,5.080700,7.900959,28.202639,8.770690,4.125697,9.029154,...,32.155294,66.929665,26.952271,1.415243,20.407581,25.332371,13.002046,12.081021,8.400113,58.815552
2,06049,20536,18.439793,16.049022,15.147598,7.618808,29.484463,3.067639,2.279369,18.865174,...,22.165539,20.770432,NaN,3.362832,26.585251,15.856325,10.921395,15.054659,17.136908,58.969287
3,06051,27321,12.011839,12.670754,11.987816,5.356805,14.718750,5.262162,17.229387,8.320748,...,21.373607,30.643653,NaN,0.834232,19.885599,25.623876,15.828839,15.828839,13.024092,70.305645
4,06053,25776,13.934287,30.863013,4.002503,6.361883,23.074733,10.889623,9.590659,10.444611,...,26.872753,65.752819,12.794056,4.290009,21.235581,26.071152,13.625486,13.042326,9.943672,62.682636


In [105]:
df = df.sort_values('fips')
df.head()

,fips,per_capita_income,percent_below_poverty,percent_no_highschool_diploma,percent_mobile_homes,percent_no_vehicle,percent_unemployed_CDC,percent_overcrowding,percent_multi_unit_housing,percent_age_65_and_older,...,percent_age_17_and_younger,percent_minorities,percent_disabled,percent_institutionalized_in_group_quarters,percent_single_parent_households_CDC,percent_age_18_to_34,percent_age_35_to_44,percent_age_45_to_54,percent_age_55_to_64,percent_age_18_to_64
385,01001,24568,10.587460,15.001180,19.941680,5.411299,19.244485,1.318592,1.806781,11.400621,...,27.702004,22.447559,NaN,0.000000,25.378608,20.536168,15.556392,14.523563,10.281253,60.897376
386,01003,26469,12.210226,12.736097,15.599940,3.317692,20.815855,1.996373,14.889260,16.491174,...,23.308360,15.922317,26.318383,1.113254,23.571045,18.878668,13.324914,14.939331,13.057551,60.200465
387,01005,15875,25.003023,29.286509,26.785859,10.862685,30.016424,1.919347,1.090667,13.740568,...,22.491787,52.788909,NaN,10.747680,46.500803,22.881692,13.863316,14.513159,12.509477,63.767645
388,01007,19918,12.639007,26.495160,32.974224,5.456256,24.703448,1.209515,1.665729,12.436975,...,23.432110,16.390977,NaN,0.650155,33.087958,22.043344,15.939850,14.489164,11.658558,64.130916
389,01009,21070,13.371317,25.779178,23.530562,3.431206,22.119029,2.751759,0.545098,14.206590,...,24.779510,10.504128,NaN,0.756721,20.180809,20.269174,14.252452,14.229521,12.262753,61.013900


In [106]:
df = df.drop(['percent_age_17_and_younger'], axis = 1)

In [107]:
df.to_csv('Social_Economic_2010.csv')